In [1]:
import pandas as pd
import sys
mylib = '/home/fra/Project/pyProj/mlib'

In [2]:
sys.path.append(mylib)

In [3]:
data_folder = "/home/fra/DataMart/datacentre/olddata/"
train_filename = 'ccbt_train.csv'

In [4]:
df = pd.read_csv(data_folder + train_filename, skipinitialspace=True)
df.head()

,target_f,GCIS_KEY,DATA_DT,CUNCZZ_AGE_YEARS,CUBCZZ_PERSONAL_CUST_IND,CUBCZZ_ALLOW_ADVERT_MAIL,CUBCZZ_ALLOW_PHONE_CONTACT,CUBCZZ_REL_MGT_IND,CUNCZZ_CUST_TENURE,CUCCZZ_CUST_GENDER,...,AFBCZZ_LOAN_OFFSET_IND,AFBCZZ_TD_IND,AFNCZZ_TD_BAL_AMT,AFBCZZ_TD_MATURE_IN_1M,AFBCZZ_TD_MATURE_IN_3M,AFBCZZ_TD_MATURE_IN_6M,AFBCZZ_FR_LOAN_MATURE_IN_3M,AFBCZZ_FR_LOAN_MATURE_IN_6M,AFBCZZ_FR_LOAN_MATURE_IN_12M,PROCESS_DTTM
0,0,55349465,31/10/2017,0,0,0,0,1,1.0,NaN,...,0,0,0.0,0,0,0,0,0,0,27:01.4
1,1,55510132,30/11/2017,33,1,0,0,0,1.0,F,...,1,0,0.0,0,0,0,0,0,0,37:10.8
2,0,50260882,30/11/2017,58,1,0,0,0,18.0,F,...,0,0,0.0,0,0,0,0,0,0,37:10.8
3,0,52934020,31/07/2017,46,1,1,1,0,9.0,F,...,0,0,0.0,0,0,0,0,0,0,07:09.2
4,0,54700818,31/05/2018,0,0,0,0,1,4.0,NaN,...,0,0,0.0,0,0,0,0,0,1,07:24.8


In [5]:
import utility as util
import explore as ex
import pipeline as pl
import transform as trf
from imp import reload

In [6]:
exclusions=['DATA_DT', 'PROCESS_DTTM', 'GCIS_KEY']
target_col = 'TARGET_F'

In [7]:
(X, y, features, py, px) = pl.binary_classfication_preprocess(df, target_col, exclusions)

In [8]:
X.shape, y.shape, features.shape

((40675, 361), (40675,), (361,))

### Out of sample: WOE

In [12]:
from sklearn.model_selection import train_test_split
(traindf, testdf) = train_test_split(df, test_size=0.3)

In [13]:
(X, y, features, py, px) = pl.binary_classfication_preprocess(traindf, target_col, exclusions)

In [14]:
import xgboost as xgb
model = xgb.XGBClassifier().fit(X, y)

In [16]:
ytest = py.transform(testdf)
Xtest = px.transform(testdf)

In [21]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix

In [18]:
def print_model_performance(model, X_test, y_test):
    predictions = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)

    print("Accuracy: %f" % accuracy_score(y_test, predictions))
    print("Confustion Matrix: ")
    print(confusion_matrix(y_true=y_test, y_pred=predictions))
    print("AUC Score: %f" % roc_auc_score(y_test, pred_prob[:,1]))
    print("GINI Coefficient: %f" % (roc_auc_score(y_test, pred_prob[:,1])*2-1))

In [22]:
print_model_performance(model, Xtest.to_numpy(), ytest)

Accuracy: 0.645333
Confustion Matrix: 
[[4171 2152]
 [2176 3704]]
AUC Score: 0.708112
GINI Coefficient: 0.416224


### Out of sample: Dummy

In [23]:
(dX, dy, dfeatures, dpy, dpx) = pl.dummy_feature_preprocess(traindf, target_col,exclusions)

In [24]:
# modelling
xgbmodel = xgb.XGBClassifier().fit(dX, dy)

In [25]:
ytest = dpy.transform(testdf)
Xtest = dpx.transform(testdf)

In [28]:
reload(util)

<module 'utility' from '/home/fra/Project/pyProj/mlib/utility.py'>

In [29]:
# prepare test data set
XX = util.reframe(Xtest, dfeatures)

In [30]:
print_model_performance(xgbmodel, XX.to_numpy(), ytest)

Accuracy: 0.545440
Confustion Matrix: 
[[1107 5216]
 [ 331 5549]]
AUC Score: 0.672046
GINI Coefficient: 0.344092


### OUt of sample: standard

In [31]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

In [40]:
ntransformer = StandardScaler
ctransformer = OrdinalEncoder
(X, y, features, py, px) = pl.standard_feature_preprocess(traindf, target_col, ntransformer, ctransformer, exclusions)

In [41]:
# modelling
xgbmodel = xgb.XGBClassifier().fit(X, y)

In [42]:
ytest = py.transform(testdf)
Xtest = px.transform(testdf)

In [43]:
XX = util.reframe(Xtest, features)

In [44]:
print_model_performance(xgbmodel, XX.to_numpy(), ytest)

Accuracy: 0.570433
Confustion Matrix: 
[[1890 4433]
 [ 809 5071]]
AUC Score: 0.660365
GINI Coefficient: 0.320730
